In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (101 kB/s

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-31 15:29:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-01-31 15:29:18 (1.21 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import pandas as pd
# Read in Zillow data from S3 Buckets
from pyspark import SparkFiles
url = "https://housingdata21.s3.us-east-2.amazonaws.com/zhvi_2018.csv"
spark.sparkContext.addFile(url)
zillow_df = spark.read.csv(SparkFiles.get("zhvi_2018.csv"), sep=",", header=True, inferSchema=True)
zillow_df.show()

+-------+----------+
|zipcode|      zhvi|
+-------+----------+
|  60002|  203155.5|
|  60004| 371788.75|
|  60005| 375617.67|
|  60007|  290179.5|
|  60008| 257214.25|
|  60010|  503737.0|
|  60012| 311768.92|
|  60013| 237906.58|
|  60014| 232307.17|
|  60015| 489269.25|
|  60016| 285147.67|
|  60018| 274593.83|
|  60020| 165690.42|
|  60021| 243928.17|
|  60022|1037845.67|
|  60025| 530058.75|
|  60026| 613297.75|
|  60029| 812012.75|
|  60030| 219686.75|
|  60031|  243986.5|
+-------+----------+
only showing top 20 rows



In [5]:
# Read in income data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/MedianIncome_Zip.csv"
spark.sparkContext.addFile(url)
income_df = spark.read.csv(SparkFiles.get("MedianIncome_Zip.csv"), sep=",", header=True, inferSchema=True)
income_df.show()

+--------+-----------------------+
|Zipcodes|Median income (dollars)|
+--------+-----------------------+
|   52761|                  55132|
|   60002|                  82692|
|   60004|                  94096|
|   60005|                  78109|
|   60007|                  79590|
|   60008|                  66462|
|   60010|                 137994|
|   60012|                 127651|
|   60013|                  98377|
|   60014|                  87256|
|   60015|                 142409|
|   60016|                  64444|
|   60018|                  60751|
|   60020|                  57518|
|   60021|                 106960|
|   60022|                 209821|
|   60025|                  97881|
|   60026|                 111875|
|   60029|                 200500|
|   60030|                  87449|
+--------+-----------------------+
only showing top 20 rows



In [6]:
# Read in ACS data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/acs_data.csv"
spark.sparkContext.addFile(url)
acs_df = spark.read.csv(SparkFiles.get("acs_data.csv"), sep=",", header=True, inferSchema=True)
acs_df.show()

+-------+---------+-----+------+-------+------+-------+--------+-------+--------+--------+--------+-----------+
|zipcode|total_pop| male|female|med_age|race_w|race_aa|race_nat|race_as|race_api|race_oth|eth_hisp|eth_nonhisp|
+-------+---------+-----+------+-------+------+-------+--------+-------+--------+--------+--------+-----------+
|  52761|    30619|15205| 15414|   37.8| 28856|   1504|     255|    493|      11|     603|    5480|      25139|
|  60002|    24066|12076| 11990|   43.3| 22792|    869|      89|    630|      31|     402|    1662|      22404|
|  60004|    50915|24193| 26722|     43| 43996|    738|      79|   5458|     116|    1597|    4013|      46902|
|  60005|    29539|15040| 14499|   43.8| 24937|    892|     143|   2961|       0|    1089|    3163|      26376|
|  60007|    33420|16504| 16916|   43.8| 28444|    532|     245|   3680|      24|    1500|    4020|      29400|
|  60008|    21833|10604| 11229|   37.4| 15923|   1179|     325|   1992|      92|    2976|    5430|     

In [7]:
# Read in crime data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/crime_data.csv"
spark.sparkContext.addFile(url)
crime_df = spark.read.csv(SparkFiles.get("crime_data.csv"), sep=",", header=True, inferSchema=True)
crime_df.show()

+-----------------+---------+------------+----+-------+-----------+----------+--------+---------+---------+-----+
|              _c0|vio_crime|murder_mansl|rape|robbery|agg_assault|prop_crime|burglary|lar_theft|car_theft|arson|
+-----------------+---------+------------+----+-------+-----------+----------+--------+---------+---------+-----+
|          Addison|       61|           1|  18|      6|         36|       464|      65|      361|       38|    4|
|           Albany|        1|           0|   1|      0|          0|         6|       2|        4|        0|    0|
|            Aledo|        0|           0|   0|      0|          0|        26|       3|       23|        0|    0|
|           Alexis|        0|           0|   0|      0|          0|        15|       7|        4|        4|    0|
|        Algonquin|       27|           0|   7|      4|         16|       265|      18|      243|        4|    0|
|            Alsip|       24|           2|   0|     16|          6|       427|      66| 

In [8]:
# Read in city data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/zipcode_to_city.csv"
spark.sparkContext.addFile(url)
city_df = spark.read.csv(SparkFiles.get("zipcode_to_city.csv"), sep=",", header=True, inferSchema=True)
city_df.show()

+--------+-----------------+
|Zip Code|             City|
+--------+-----------------+
|   60001|            Alden|
|   60002|          Antioch|
|   60004|Arlington Heights|
|   60005|Arlington Heights|
|   60006|Arlington Heights|
|   60007|Elk Grove Village|
|   60008|  Rolling Meadows|
|   60009|Elk Grove Village|
|   60010|       Barrington|
|   60011|       Barrington|
|   60012|     Crystal Lake|
|   60013|             Cary|
|   60014|     Crystal Lake|
|   60015|        Deerfield|
|   60016|      Des Plaines|
|   60017|      Des Plaines|
|   60018|      Des Plaines|
|   60019|      Des Plaines|
|   60020|         Fox Lake|
|   60021|  Fox River Grove|
+--------+-----------------+
only showing top 20 rows



In [9]:
zillow_df = zillow_df.toPandas()
income_df = income_df.toPandas()
acs_df = acs_df.toPandas()
crime_df = crime_df.toPandas()
city_df = city_df.toPandas()

In [10]:
income_df = income_df.rename(columns = {"Median income (dollars)": "median_income", "Zipcodes": "zipcode"})
income_df

,zipcode,median_income
0,52761,55132
1,60002,82692
2,60004,94096
3,60005,78109
4,60007,79590
...,...,...
788,61857,32500
789,61858,60068
790,61859,72344
791,61862,71827


In [11]:
city_df = city_df.rename(columns = {"Zip Code": "zipcode", "City": "city"})
city_df

,zipcode,city
0,60001,Alden
1,60002,Antioch
2,60004,Arlington Heights
3,60005,Arlington Heights
4,60006,Arlington Heights
...,...,...
1582,62995,Vienna
1583,62996,Villa Ridge
1584,62997,Willisville
1585,62998,Wolf Lake


In [12]:
crime_df = crime_df.rename(columns = {"_c0": "city"})
crime_df

,city,vio_crime,murder_mansl,rape,robbery,agg_assault,prop_crime,burglary,lar_theft,car_theft,arson
0,Addison,61,1,18,6,36,464,65,361,38,4.0
1,Albany,1,0,1,0,0,6,2,4,0,0.0
2,Aledo,0,0,0,0,0,26,3,23,0,0.0
3,Alexis,0,0,0,0,0,15,7,4,4,0.0
4,Algonquin,27,0,7,4,16,265,18,243,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
516,Woodstock,24,0,10,2,12,298,27,266,5,0.0
517,Worth,26,0,11,7,8,123,21,92,10,1.0
518,Yates City,1,0,1,0,0,8,1,7,0,0.0
519,Yorkville,18,0,8,3,7,200,17,178,5,2.0


In [13]:
income_df = income_df[income_df.median_income != "-"]
income_df

,zipcode,median_income
0,52761,55132
1,60002,82692
2,60004,94096
3,60005,78109
4,60007,79590
...,...,...
788,61857,32500
789,61858,60068
790,61859,72344
791,61862,71827


In [14]:
income_df[["median_income"]] = income_df[["median_income"]].apply(pd.to_numeric)
income_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,zipcode,median_income
0,52761,55132
1,60002,82692
2,60004,94096
3,60005,78109
4,60007,79590
...,...,...
788,61857,32500
789,61858,60068
790,61859,72344
791,61862,71827


In [15]:
acs_df = acs_df.replace(',','', regex=True)

In [17]:
acs_df = acs_df[acs_df != "-"]
acs_df

,zipcode,total_pop,male,female,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp
0,52761,30619,15205,15414,37.8,28856,1504,255,493,11,603,5480,25139
1,60002,24066,12076,11990,43.3,22792,869,89,630,31,402,1662,22404
2,60004,50915,24193,26722,43,43996,738,79,5458,116,1597,4013,46902
3,60005,29539,15040,14499,43.8,24937,892,143,2961,0,1089,3163,26376
4,60007,33420,16504,16916,43.8,28444,532,245,3680,24,1500,4020,29400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,62996,464,205,259,51.3,387,123,35,0,0,0,0,464
1381,62997,598,314,284,33.5,591,0,0,7,0,0,0,598
1382,62998,323,167,156,35.9,323,0,0,0,0,0,0,323
1383,62999,1794,915,879,32.8,1786,6,29,7,0,29,34,1760


In [18]:
acs_df = acs_df.apply(pd.to_numeric)
acs_df

,zipcode,total_pop,male,female,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp
0,52761,30619,15205,15414,37.8,28856,1504,255,493,11,603,5480,25139
1,60002,24066,12076,11990,43.3,22792,869,89,630,31,402,1662,22404
2,60004,50915,24193,26722,43.0,43996,738,79,5458,116,1597,4013,46902
3,60005,29539,15040,14499,43.8,24937,892,143,2961,0,1089,3163,26376
4,60007,33420,16504,16916,43.8,28444,532,245,3680,24,1500,4020,29400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,62996,464,205,259,51.3,387,123,35,0,0,0,0,464
1381,62997,598,314,284,33.5,591,0,0,7,0,0,0,598
1382,62998,323,167,156,35.9,323,0,0,0,0,0,0,323
1383,62999,1794,915,879,32.8,1786,6,29,7,0,29,34,1760


In [19]:
crime_df = crime_df.replace(',','', regex=True)

In [20]:
crime_df[["vio_crime", "murder_mansl", "rape", "robbery", "agg_assault", "prop_crime", "burglary", "lar_theft", "car_theft", "arson"]] = crime_df[["vio_crime", "murder_mansl", "rape", "robbery", "agg_assault", "prop_crime", "burglary", "lar_theft", "car_theft", "arson"]].apply(pd.to_numeric)
crime_df

,city,vio_crime,murder_mansl,rape,robbery,agg_assault,prop_crime,burglary,lar_theft,car_theft,arson
0,Addison,61.0,1,18,6,36.0,464.0,65.0,361.0,38.0,4.0
1,Albany,1.0,0,1,0,0.0,6.0,2.0,4.0,0.0,0.0
2,Aledo,0.0,0,0,0,0.0,26.0,3.0,23.0,0.0,0.0
3,Alexis,0.0,0,0,0,0.0,15.0,7.0,4.0,4.0,0.0
4,Algonquin,27.0,0,7,4,16.0,265.0,18.0,243.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
516,Woodstock,24.0,0,10,2,12.0,298.0,27.0,266.0,5.0,0.0
517,Worth,26.0,0,11,7,8.0,123.0,21.0,92.0,10.0,1.0
518,Yates City,1.0,0,1,0,0.0,8.0,1.0,7.0,0.0,0.0
519,Yorkville,18.0,0,8,3,7.0,200.0,17.0,178.0,5.0,2.0


In [21]:
zillow_df = spark.createDataFrame(zillow_df)
income_df = spark.createDataFrame(income_df)
acs_df = spark.createDataFrame(acs_df)
crime_df = spark.createDataFrame(crime_df)
city_df = spark.createDataFrame(city_df)

In [27]:
city_df.dtypes

[('zipcode', 'bigint'), ('city', 'string')]

In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [28]:
# Write DataFrame to zillow_data table in RDS
zillow_df.write.jdbc(url=jdbc_url, table='zhvi_data', mode=mode, properties=config)

In [29]:
# Write DataFrame to income_data table in RDS
income_df.write.jdbc(url=jdbc_url, table='income_data', mode=mode, properties=config)

In [30]:
# Write DataFrame to acs_data table in RDS
acs_df.write.jdbc(url=jdbc_url, table='acs_data', mode=mode, properties=config)

In [32]:
# Write DataFrame to crime_data table in RDS
crime_df.write.jdbc(url=jdbc_url, table='crime_data', mode=mode, properties=config)

In [33]:
# Write DataFrame to city_data table in RDS
city_df.write.jdbc(url=jdbc_url, table='city_data', mode=mode, properties=config)